# Create Indexed GRE Coincidental Peak
Creates start and stop indexes the coincidental load.

In [0]:
%run ../Utilities/ConfigUtilities

In [0]:
# Set up the environment using a function in ConfigUtilties.
set_spark_config()

In [0]:
from pyspark.sql.functions import col

debug = 1

In [0]:
# Read the monthly coincidental peak.
coincid_df = spark.read.format("delta").load(COINCIDENTAL_LOAD_SILVER_PATH)

if debug:
    display(coincid_df)

In [0]:
# Read the indexed calendar
cal_df =  spark.read.parquet(INDEXED_CALENDAR_PATH)

# Coincidental peka data is local time zone based, so drop the UTC columns for this process.
cal_df = cal_df.drop("UTCTimeStamp", "UTCYear", "UTCMonth", "UTCDay", "UTCHour", "UTCMinute")

if debug:
    display(cal_df)

In [0]:
# Join based on timestamps.
coincid_cal_df = coincid_df.join(cal_df, coincid_df.CoincidTime == cal_df.LocalTimeStamp, "inner")

if debug:
    display(coincid_cal_df)

In [0]:
# Drop the Local columns and save.
coincid_cal_df = coincid_cal_df.drop("LocalTimeStamp", "LocalYear", "LocalMonth", "LocalDay", "LocalHour", "LocalMinute", "Holiday", "Weekend")

coincid_cal_df.write.format("delta").mode("overwrite").save(COINCIDENTAL_LOAD_INDEX_PATH)

In [0]:
# Clean up the delta history.
spark.sql(f"VACUUM '{COINCIDENTAL_LOAD_INDEX_PATH}'")